<a href="https://colab.research.google.com/github/gillesretiere/langdeck/blob/novomaster/langdeck_pipe_2023_v2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import pandas as pd
pd.set_option("display.width",1000)
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
import re
import string
import unicodedata
import ast

!pip3 install pyclean

# Gspread  API for Google sheets
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall -y gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting

#==== Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/langdeck/
from google.colab import files
from google.colab import auth
auth.authenticate_user()

#====== Google Credentials ================
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()

# python .env see -> https://saurabh-kumar.com/python-dotenv/
!pip install python-dotenv
from dotenv import load_dotenv
from dotenv import dotenv_values
load_dotenv()
dotenv_config = dotenv_values(".env")
app_path = dotenv_config["APP_PATH"]

#====== GSpread is a Python API for Google Sheets.================
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
gc = gspread.authorize(creds)

#====== Airtable ================
!pip install airtable-python-wrapper
from airtable import Airtable

#====== Deep Translate ================
!pip install -U deep-translator
!pip install --upgrade deepl

#====== Dictionaries ================
# larousse API
!pip install larousse-api-sunbro

#====== Lemmatizers ================
# https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

#====== Text To Speech
# about quotas : https://stackoverflow.com/questions/65980562/gtts-tts-gttserror-429-too-many-requests-from-tts-api-probable-cause-unknow
import os
!pip install gtts --upgrade
from gtts import gTTS

# CDN Cloudinary for upload
!pip3 install cloudinary
import cloudinary
import cloudinary.uploader
import cloudinary.api
from cloudinary.uploader import upload
from cloudinary.utils import cloudinary_url

!export PYTHONIOENCODING=utf8

%load_ext autoreload
%autoreload 2
%cd /content/drive/MyDrive/langdeck/
!pyclean .
import sys, os
sys.path.append('/content/drive/MyDrive/langdeck/py_modules')
import py_modules
py_modules.test()

# .env
from dotenv import load_dotenv
from dotenv import dotenv_values
%cd /content/drive/MyDrive/langdeck/
load_dotenv()
dotenv_config = dotenv_values(".env")

# Config CDN
cloudinary.config(
  cloud_name = dotenv_config["CLOUDINARY_CLOUD_NAME"],
  api_key = dotenv_config["CLOUDINARY_API_KEY"],
  api_secret = dotenv_config["CLOUDINARY_API_SECRET"],
  secure = True
)

load_dotenv()
dotenv_config = dotenv_values(".env")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 778.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 721.1 kB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Found existing installation: gspread-dataframe 3.3.1
Uninstalling gspread-dataframe-3.3.1:
  Successfully uninstalled gspread-dataframe-3.3.1
Mounted at /content/drive
/content/drive/MyDrive/langdeck
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 991.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=1f49bfd6efd8ca7f88fa7dd02d6d7d9161f4310d0ab8eb295ecc24db3321e7e6
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Cloning https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to /tmp/pip-re

# Test cnx integration DB
Ici on teste la cnx avec la base d'intégration qui est une table GSHEET

In [ ]:
def deep_translator_translate (input, source_language, target_language):
  from deep_translator import GoogleTranslator
  return GoogleTranslator(source=source_language, target=target_language).translate(input)

In [2]:
db_lgdk_intgr = dotenv_config["DB_LGDK_INTGR"]
#--- integration database
sheet = db_lgdk_intgr
wb_intgr  = gc.open_by_url(sheet)
#--- Liste des codes ISO639-3
df_lang_alpha3 = py_modules.load_df_from_gsheet (wb_intgr, "ref_iso-639-3")
#--- Liste des langues (issues d'un Corpus)
df_lang_corpus = py_modules.load_df_from_gsheet (wb_intgr, "ref_lang_corpus")
# df_lang_corpus.head()

# Test cnx DSGN DB
Ici on teste la cnx avec la base de DESIGN qui est une table Airtable
##### doc : https://airtable-python-wrapper.readthedocs.io/en/airtable-python-wrapper/index.html


In [3]:
#---- Base Airtable 2023
api_key_airtable = dotenv_config["AIRTABLE_PIPE23_API_KEY"]
headers = {"Authorization": "Bearer " + api_key_airtable,"Content-Type" : "application/json"}
base_id = dotenv_config["AIRTABLE_PIPE23_BASE_ID"]
table_name = dotenv_config["AIRTABLE_PIPE23_TBL_NAME"]
airtable = Airtable(base_id, table_name, api_key_airtable)
active_filter = "{Status}='Done'"

# Langues

In [4]:
tbd_languages = Airtable(base_id, "languages", api_key_airtable)
lang_filter = "{lang_is_available}='TRUE'"
vw_lang_grid = tbd_languages.get_all(
    view='view_grid',
    sort=['lang_id','lang_is_available'],
    formula=lang_filter,
    )

# Thèmes

## Intégration

In [5]:
tbd_themes = Airtable(base_id, "themes", api_key_airtable)
vw_themes_grid = tbd_themes.get_all(
    view='view_grid',
    sort=['theme_name'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
df_themes = pd.DataFrame.from_records((r['fields'] for r in vw_themes_grid))
df_themes = df_themes[["theme_rec_id",
                 "theme_name",
                 "theme_desc",
                 "stories",
                 ]]


#=======================================================
# Ajout des langues en colonnes
#=======================================================
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
py_modules.save_df_to_gsheet (wb_intgr, "tbl_languages", df_languages)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_themes = pd.concat([df_themes, df_t.reindex(df_themes.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_themes,
        id_vars=["theme_rec_id",
                 "theme_name",
                 "theme_desc",
                 "stories"],
        var_name="theme_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["theme_translation_id"] = df_w_temp[['theme_rec_id', 'theme_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)


#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_themes = py_modules.load_df_from_gsheet (wb_intgr, "tbl_themes")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_themes[["theme_translation_id",
                  "theme_translation",
                  "theme_translation_status",
                  "theme_audio",
                  "theme_audio_url",
                  "theme_audio_url_fr"]],
    on="theme_translation_id",
    how="left")

## Traduction

In [6]:
df_w_tr["theme_translation"] = df_w_tr["theme_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["theme_translation_status"] = df_w_tr["theme_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["theme_translation","theme_translation_status"]] = df_w_tr[["theme_translation",
                                                                   "theme_name",
                                                                   "theme_language",
                                                                   "theme_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["theme_audio"] = df_w_tr[["theme_translation_id","theme_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

## Text To Speech

In [7]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["theme_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["theme_translation","theme_language","theme_audio","theme_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['theme_audio_url'] = df_tts['theme_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["theme_audio_url"] = df_tts[["theme_language","theme_audio","theme_audio_url","theme_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["theme_audio_url_fr"] = df_tts["theme_rec_id"].map(
    lambda x:py_modules.theme_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


In [8]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_themes", df_tts)

# Stories

## Intégration

In [9]:
tbd_stories = Airtable(base_id, "stories", api_key_airtable)
vw_stories_grid = tbd_stories.get_all(
    view='view_grid',
    sort=['story_name','story_level'],
    formula=active_filter,
    )
#=======================================================
# Transfo en dataframe
#=======================================================
df_stories = pd.DataFrame.from_records((r['fields'] for r in vw_stories_grid))
df_stories.rename(columns={'story_theme': 'story_theme_rec_id'}, inplace=True)
df_stories["story_theme"] = df_stories["story_theme_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_themes, x, "theme_name"),
    na_action="ignore"
)
vk_col_stories = ["story_rec_id",
                 "story_name",
                 "story_episode",
                 "story_level",
                 "story_theme",
                 "story_illustration",
                 "paragraphs",
                 "story_desc",
                 ]
df_stories = df_stories[vk_col_stories]
#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_stories = pd.concat([df_stories, df_t.reindex(df_stories.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_stories,
        id_vars=vk_col_stories,
        var_name="story_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["story_translation_id"] = df_w_temp[['story_rec_id', 'story_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_stories = py_modules.load_df_from_gsheet (wb_intgr, "tbl_stories")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_stories[["story_translation_id",
                  "story_translation",
                  "story_translation_status",
                  "story_audio",
                  "story_audio_url",
                  "story_audio_url_fr",
                  "story_desc_translation",
                  ]],
    on="story_translation_id",
    how="left")

## Traduction

In [10]:
df_w_tr["story_translation"] = df_w_tr["story_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["story_translation_status"] = df_w_tr["story_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["story_translation","story_translation_status"]] = df_w_tr[["story_translation",
                                                                   "story_name",
                                                                   "story_language",
                                                                   "story_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["story_audio"] = df_w_tr[["story_translation_id","story_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

## Text To Speech

In [11]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["story_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["story_translation","story_language","story_audio","story_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['story_audio_url'] = df_tts['story_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["story_audio_url"] = df_tts[["story_language","story_audio","story_audio_url","story_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["story_audio_url_fr"] = df_tts["story_rec_id"].map(
    lambda x:py_modules.story_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


### Translation of story description

In [12]:
# translate if necessary # status == NaN OR START
df_tts["story_desc_translation"] = df_tts["story_desc_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_tts[["story_desc_translation","story_desc_translation_status"]] = df_tts[
    ["story_desc_translation",
     "story_desc",
     "story_language",
     "story_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='Start')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


In [13]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_stories", df_tts)

# Words

In [18]:
tbd_words = Airtable(base_id, "words", api_key_airtable)
vw_words_grid = tbd_words.get_all(
    view='view_grid',
    sort=['word','word_type'],
    formula=active_filter,
    )

## Intégration

In [19]:
#=======================================================
# Transfo en dataframe
#=======================================================
df_words = pd.DataFrame.from_records((r['fields'] for r in vw_words_grid))
# on ne garde que les colonnes utiles
df_words = df_words[["word_rec_id",
                     "word",
                     "word_type",
                     "word_lemmas",
                     "word_notes",
                     "word_nutri_notes",
                     "word_notes_status",
                     "word_nutri_notes_status",
                     ]]
# Capitalisation
df_words["word"]=df_words["word"].map(lambda x:x[:1].upper() + x[1:])
# on convertit les rec_id en valeur litérale
df_words.rename(columns={'word_lemmas': 'word_lemmas_rec_id'}, inplace=True)
# on convertit les rec_id en valeur litérale
df_words["word_lemmas"] = df_words["word_lemmas_rec_id"].map(
    lambda x:py_modules.get_word_lemma_by_id(tbd_words, x),
    na_action="ignore")

#=======================================================
# Ajout des langues en colonnes
#=======================================================
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_words = pd.concat([df_words, df_t.reindex(df_words.index)], axis=1)

#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_words,
        id_vars=["word_rec_id",
                 "word",
                 "word_type",
                 "word_lemmas",
                 "word_lemmas_rec_id",
                 "word_notes",
                 "word_nutri_notes",
                 "word_notes_status",
                 "word_nutri_notes_status",
                 ],
        var_name="word_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["word_translation_id"] = df_w_temp[['word_rec_id', 'word_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_words = py_modules.load_df_from_gsheet (wb_intgr, "tbl_words")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_words[["word_translation_id",
                  "word_translation",
                  "word_translation_status",
                  "word_audio",
                  "word_audio_url",
                  "word_audio_url_fr",
                  "word_notes_translation",
                  "word_notes_translation_status",
                  "word_nutri_notes_translation",
                  "word_nutri_notes_translation_status",
                  ]],
    on="word_translation_id",
    how="left")


## Traduction

In [20]:
df_w_tr["word_translation"] = df_w_tr["word_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_translation_status"] = df_w_tr["word_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# il faut forcer la maj car les lemmes ont peut-être changé => on recrée la colonne from scratch
df_w_tr["word_lemmas_tr_rec_id"]=df_w_tr[["word_lemmas_rec_id","word_language"]].apply(
    lambda x:py_modules.f_word_lemmas_tr_rec_id(x[0],x[1]) if type(x[0])==list else np.nan,
    axis=1)
# translate if necessary # status == NaN OR START
df_w_tr[["word_translation","word_translation_status"]] = df_w_tr[["word_translation",
                                                                   "word",
                                                                   "word_language",
                                                                   "word_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# on convertit les rec_id en valeur litérale
df_w_tr["word_lemmas_tr"] = df_w_tr["word_lemmas_tr_rec_id"].map(
    lambda x:py_modules.f_word_lemmas_tr(x, df_w_tr),
    na_action="ignore")

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["word_audio"] = df_w_tr[["word_translation_id","word_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)



Translated Super... from fre to nl >>> Fantastisc...
Translated Super... from fre to en >>> Great...
Translated Super... from fre to fr >>> Super...
Translated Super... from fre to de >>> Großartig...


## Notes and Nutri notes

In [21]:
df_w_tr["word_notes_translation"] = df_w_tr["word_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_notes_translation_status"] = df_w_tr["word_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

In [22]:
# notes et nutri-notes
# translate if necessary # status == NaN OR START
df_w_tr[["word_notes_translation","word_notes_translation_status"]] = df_w_tr[
    ["word_notes_translation",
     "word_notes",
     "word_language",
     "word_notes_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


Translated Interjecti... from fre to nl >>> Tussenwerp...
Translated Interjecti... from fre to en >>> Interjecti...
Translated Interjecti... from fre to fr >>> Interjecti...
Translated Interjecti... from fre to de >>> Zwischenru...


In [23]:
df_w_tr["word_nutri_notes_translation"] = df_w_tr["word_nutri_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_nutri_notes_translation_status"] = df_w_tr["word_nutri_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

In [24]:
# notes et nutri-notes
# translate if necessary # status == NaN OR START
df_w_tr[["word_nutri_notes_translation","word_nutri_notes_translation_status"]] = df_w_tr[
    ["word_nutri_notes_translation",
     "word_nutri_notes",
     "word_language",
     "word_nutri_notes_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


## Text To Speech

In [25]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["word_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["word_translation","word_language","word_audio","word_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x[1] + "/" + x[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
or x[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['word_audio_url'] = df_tts['word_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["word_audio_url"] = df_tts[["word_language","word_audio","word_audio_url","word_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2]) or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["word_audio_url_fr"] = df_tts["word_rec_id"].map(
    lambda x:py_modules.word_get_audio_url_fr(x, df_tts))

py_modules.save_df_to_gsheet (wb_intgr, "tbl_words", df_tts)

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger
/content/drive/MyDrive/langdeck/assets/audio/ai/dut/recVyn4uxhrR2JuNa-dut.mp3(nl) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/eng/recVyn4uxhrR2JuNa-eng.mp3(en) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/fre/recVyn4uxhrR2JuNa-fre.mp3(fr) : okay
/content/drive/MyDrive/langdeck/assets/audio/ai/ger/recVyn4uxhrR2JuNa-ger.mp3(de) : okay
iso3:dut ; local_folder:/content/drive/MyDrive/langdeck/assets/audio/ai/dut/ ; path:recVyn4uxhrR2JuNa-dut.mp3
http://res.cloudinary.com/dhc7ovnwk/video/upload/v1700130228/langdeck/assets/audio/ai/dut/recVyn4uxhrR2JuNa-dut.mp3
iso3:eng ; local_folder:/content/drive/MyDrive/langdeck/assets/audio/ai/eng/ ; path:recVyn4uxhrR2JuNa-eng.mp3
http://res.cloudinary.com/dhc7ovnwk/video/upload/v1700130229/langdeck/assets/audio/ai/eng/recVyn4uxh

# Phrases

## Intégration

In [26]:
# cette fonction remplace le markdown rtf employé par Airtable par un tag HTML paramétrable
def phrase_words_markdown_to_html_kw (source, id):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'<kw>', temp)
  target = re.sub(r'\`\*+', '<kw>', temp)
  return target

# cette fonction remplace l'id temporaire par l'id du mot clé
def phrase_words_span_rec_id_kw(source, vk_id):
  from itertools import count
  counter = count(0)
  target = re.sub(r'<kw>(.+?)<kw>', lambda x: '<kw>' + vk_id[next(counter)] + '<kw>', source)
  return target

In [27]:
tbd_phrases = Airtable(base_id, "phrases", api_key_airtable)
vw_phrases_grid = tbd_phrases.get_all(
    view='view_grid',
    sort=['phrase_related_story','phrase_paragraph','phrase_position'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
df_phrases = pd.DataFrame.from_records((r['fields'] for r in vw_phrases_grid))
df_phrases.rename(columns={'phrase_words': 'phrase_words_rec_id'}, inplace=True)
df_phrases["phrase_words"] = df_phrases["phrase_words_rec_id"].map(
    lambda x:py_modules.get_word_lemma_by_id(tbd_words, x),
    na_action="ignore")
# récupérer la saynète à partir de son id
df_phrases.rename(columns={'phrase_related_story': 'phrase_related_story_rec_id'}, inplace=True)
df_phrases["phrase_related_story"] = df_phrases["phrase_related_story_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_stories, x, "story_name"),
    na_action="ignore")

df_phrases["phrase_html"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html(x[0], x[1]),
    axis=1)

df_phrases["phrase_html"] = df_phrases[["phrase_html","phrase_words_rec_id"]].apply(
    lambda x:py_modules.phrase_words_span_rec_id(x[0], x[1])
    if type(x[1])==list
    else np.nan,
    axis=1)

df_phrases["phrase_html_rec_id"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:phrase_words_markdown_to_html_kw(x[0], x[1]),
    axis=1)

df_phrases["phrase_html_rec_id"] = df_phrases[["phrase_html_rec_id","phrase_words_rec_id"]].apply(
    lambda x:phrase_words_span_rec_id_kw(x[0], x[1])
    if type(x[1])==list
    else np.nan,
    axis=1)

df_phrases["phrase_html_kw"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:phrase_words_markdown_to_html_kw(x[0], x[1]),
    axis=1)

In [28]:
vk_col_phrases = [
    "phrase_rec_id",
    "phrase",
    "phrase_words",
    "phrase_words_rec_id",
    "phrase_html",
    "phrase_html_rec_id",
    "phrase_html_kw",
    "phrase_related_story",
    "phrase_related_story_rec_id",
    "phrase_paragraph",
    "phrase_position",
    "phrase_status",
    "phrase_notes",
    "phrase_notes_2",
    "phrase_notes_status",
    "phrase_notes_2_status",
    ]
df_phrases = df_phrases[vk_col_phrases]

#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_phrases = pd.concat([df_phrases, df_t.reindex(df_phrases.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_phr_temp = pd.melt(df_phrases,
        id_vars = vk_col_phrases,
        var_name = "phrase_language",
        value_name = "translation")
# hash incl. language iso3
df_phr_temp["phrase_translation_id"] = df_phr_temp[['phrase_rec_id', 'phrase_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
# left join to include translations
df_phr_temp.drop(columns=["translation",], axis=1, inplace=True)

#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_phrases = py_modules.load_df_from_gsheet (wb_intgr, "tbl_phrases")
df_phr_tr = pd.merge(
    df_phr_temp,
    df_tbl_phrases[[
        "phrase_translation_id",
        "phrase_translation",
        "phrase_translation_status",
        "phrase_audio",
        "phrase_audio_url",
        "phrase_audio_url_fr",
        "phrase_notes_translation",
        "phrase_notes_translation_status",
        "phrase_notes_2_translation",
        "phrase_notes_2_translation_status",
        ]],
    on="phrase_translation_id",
    how="left")

In [29]:
df_phr_tr["phrase_notes_translation"] = df_phr_tr["phrase_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_translation_status"] = df_phr_tr["phrase_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_2_translation"] = df_phr_tr["phrase_notes_2_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_2_translation_status"] = df_phr_tr["phrase_notes_2_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

## Traduction

### Traductions : mots à ne pas traduire
En attendant d'avoir les fonctionnalités de DeepL, on ignore les tags
```
def phrase_notes_markdown_to_html_kw (source):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'<span translate="no">', temp)
  target = re.sub(r'\`\*+', '</span>', temp)
  return target
  
def phrase_notes_markdown_to_html_kw_2 (source):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'<code>', temp)
  target = re.sub(r'\`\*+', '</code>', temp)
  return target
```



In [30]:
# cette fonction remplace le markdown rtf employé par Airtable par un tag HTML paramétrable
def phrase_notes_markdown_to_html_kw_2 (source):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'', temp)
  target = re.sub(r'\`\*+', '', temp)
  return target

In [31]:
df_phr_tr["phrase_notes"] = df_phr_tr["phrase_notes"].map(lambda x:phrase_notes_markdown_to_html_kw_2(x) if not pd.isna(x) else np.nan)
df_phr_tr["phrase_notes_2"] = df_phr_tr["phrase_notes_2"].map(lambda x:phrase_notes_markdown_to_html_kw_2(x) if not pd.isna(x) else np.nan)

In [32]:
# notes
# translate if necessary # status == NaN OR START
df_phr_tr[["phrase_notes_translation","phrase_notes_translation_status"]] = df_phr_tr[
    ["phrase_notes_translation",
     "phrase_notes",
     "phrase_language",
     "phrase_notes_translation_status",
     "phrase_notes_status",
     ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='Start' or x[4]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


In [33]:
# notes
# translate if necessary # status == NaN OR START
df_phr_tr[["phrase_notes_2_translation","phrase_notes_2_translation_status"]] = df_phr_tr[
    ["phrase_notes_2_translation",
     "phrase_notes_2",
     "phrase_language",
     "phrase_notes_2_translation_status",
     "phrase_notes_2_status",
     ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if ((pd.isna(x[0]) and not pd.isna(x[1])) or x[3]=='Start' or x[4]=='update')
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )


In [34]:
df_phr_tr["phrase_translation"] = df_phr_tr["phrase_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_translation_status"] = df_phr_tr["phrase_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_phr_tr[["phrase_translation","phrase_translation_status"]] = df_phr_tr[[
    "phrase_translation",
    "phrase",
    "phrase_language",
    "phrase_translation_status",
    "phrase_status",
    ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x[1],
        "fre",
        x[2],
        x[3],
        df_languages)
    if (pd.isna(x[0]) or pd.isna(x[3]) or x[3]=='Start' or x[4]=="update")
    else (x[0],x[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_phr_tr["phrase_audio"] = df_phr_tr[["phrase_translation_id","phrase_language"]].apply(
    lambda x:"assets/audio/ai/"+x[1]+"/"+str(x[0])+".mp3",
    axis=1)

## Text To Speech

In [35]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x.upper()=="TRUE" else False)
vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_phr_tr.loc[df_phr_tr["phrase_language"].isin(vk_tts)]
df_tts["phrase_translation_status"] = df_tts[["phrase_translation_status","phrase_status"]].apply(
    lambda x:
    x[1] if x[1].lower() == "update" else x[0],
    axis=1)
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[[
    "phrase_translation",
    "phrase_language",
    "phrase_audio",
    "phrase_translation_status",
    ]].apply(
    lambda x:py_modules.text2speech(
      x[0],
      df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
      audio_path + x[1] + "/" + x[2].split("/")[-1],
      False
    )
    if py_modules.is_audio(audio_path + x[1] + "/", x[2].split("/")[-1])==False
      or x[3].lower()=="update"
    else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['phrase_audio_url'] = df_tts['phrase_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["phrase_audio_url"] = df_tts[[
    "phrase_language",
    "phrase_audio",
    "phrase_audio_url",
    "phrase_translation_status",
    ]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x[0],
    audio_path + x[0]+"/",
    x[1].split("/")[-1],
    x[3])
    if pd.isna(x[2])
      or x[3].lower() == "update"
    else x[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["phrase_audio_url_fr"] = df_tts["phrase_rec_id"].map(
    lambda x:py_modules.phrase_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/dut
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/ger


In [36]:
#=======================================================
# TRADUCTION DES MOTS ASSOCIES
#=======================================================
df_tbl_stories = py_modules.load_df_from_gsheet (wb_intgr, "tbl_stories")
df_tbl_words = py_modules.load_df_from_gsheet (wb_intgr, "tbl_words")
# # construit un id concaténé avec le code langue pour chaque element du vecteur
df_tts["phrase_words_tr_rec_id"] = df_tts[["phrase_words_rec_id","phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x[0],
        x[1])
    if type(x[0])==list
    else np.nan,
    axis=1)
# récupère le mot traduit depuis son identifiant
df_tts["phrase_words_tr"] = df_tts["phrase_words_tr_rec_id"].map(
    lambda x:py_modules.set_phrase_words_tr(x, df_tbl_words),
    na_action='ignore')
#=======================================================
# TRADUCTION DES META DONNEES (STORY, THEME, ETC.)
#=======================================================
df_tts["phrase_related_story_tr_rec_id"] = df_tts[["phrase_related_story_rec_id","phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x[0],
        x[1])
    if type(x[0])==list
    else np.nan,
    axis=1)

# récupère le mot traduit depuis son identifiant
df_tts["phrase_related_story_tr"] = df_tts["phrase_related_story_tr_rec_id"].map(
    lambda x:py_modules.get_story_tr(x, df_tbl_stories),
    na_action='ignore')

In [37]:
# on rebascule les états si il y a eu update
df_tts["phrase_translation_status"] = df_tts[["phrase_translation_status","phrase_status"]].apply(
    lambda x:
    "Pending" if x[1].lower() == "update" else x[0],
    axis=1)

py_modules.save_df_to_gsheet (wb_intgr, "tbl_phrases", df_tts)

# Transformation format JSON

## Stories

In [62]:
df_stories = py_modules.load_df_from_gsheet(wb_intgr, "tbl_stories")
df_stories.sort_values (
    by=["story_language",
        "story_level",
        "story_name",
        "story_episode",],
    inplace=True)
vk_stories_cols = [
    "story_rec_id",
    "story_translation_id",
    "story_name",
    "story_translation",
    "story_level",
    "story_episode",
    "story_audio_url",
    "story_audio_url_fr",
    "story_illustration",
    "story_desc",
    "story_desc_translation",
    ]
vk_stories_vals = df_stories.groupby(["story_rec_id"])[vk_stories_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Phrases

In [63]:
df_phrases = py_modules.load_df_from_gsheet(wb_intgr, "tbl_phrases") # load source sheet
df_phrases["phrase_related_story"] = df_phrases["phrase_related_story"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_rec_id"] = df_phrases["phrase_related_story_rec_id"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_tr"] = df_phrases["phrase_related_story_tr"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_tr_rec_id"] = df_phrases["phrase_related_story_tr_rec_id"].map(lambda x:ast.literal_eval(x)[0])
df_themes = py_modules.load_df_from_gsheet(wb_intgr, "tbl_themes") # load source sheet
df_phrases.sort_values(by=['phrase_language','phrase_related_story','phrase_paragraph','phrase_position'], inplace=True)

# merge phrases + story => vue à plat avec les données étendues
df_phr_ext = pd.merge(df_phrases, df_stories[vk_stories_cols],
         left_on="phrase_related_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of columns
vk_phrases_cols = [
    "phrase_language",
    "phrase_related_story",
    "phrase_related_story_rec_id",
    "phrase_paragraph",
    "phrase_position",
    "phrase_rec_id",
    "phrase",
    "phrase_words",
    "phrase_words_rec_id",
    "phrase_words_tr",
    "phrase_words_tr_rec_id",
    "phrase_html",
    "phrase_html_rec_id",
    "phrase_html_kw",
    "phrase_translation",
    "phrase_audio",
    "phrase_audio_url",
    "phrase_audio_url_fr",
    "phrase_related_story_tr",
    "phrase_notes",
    "phrase_notes_2",
    "phrase_notes_translation",
    "phrase_notes_2_translation",
    "story_rec_id",
    "story_translation_id",
    "story_name",
    "story_translation",
    "story_level",
    "story_episode",
    "story_audio_url",
    "story_audio_url_fr",
    "story_illustration",
    "story_desc",
    "story_desc_translation",
    ] # fields (columns)

# vector of values grouped by language
vk_phrases_vals = df_phr_ext.groupby(["phrase_language"])[vk_phrases_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()


## Words and keywords

In [64]:
df_words = py_modules.load_df_from_gsheet(wb_intgr, "tbl_words")
df_words["order"] = df_words["word_type"].map(lambda x:1 if x=="Lemma" else 2 if x=="Expression" else 3)
df_words.sort_values(by=['word_language','word','order',], inplace=True)
vk_word_cols = [
    "word_language",
    "word",
    "word_rec_id",
    "order",
    "word_type",
    "word_translation_id",
    "word_translation",
    "word_lemmas",
    "word_lemmas_rec_id",
    "word_audio",
    "word_audio_url",
    "word_audio_url_fr",
    "word_lemmas_tr_rec_id",
    "word_lemmas_tr",
    "word_notes",
    "word_nutri_notes",
    "word_notes_translation",
    "word_nutri_notes_translation",
    ] # fields (columns)

vk_word_vals = df_words.groupby(["word_language"])[vk_word_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Keywords

In [65]:
def f_phrases_list_to_dict_by_col(col_name, vkp):
  vkf = list(dict.fromkeys(list(map(
        lambda x: x[col_name],
        filter(lambda d: d['phrase_rec_id'] in vkp,
               sorted(vk_lang_dict, key=lambda x: x["phrase_paragraph"]))))))[0]
  return vkf

def f_words_get_value_by_col (df, col_key, col_name, key):
  cval = df.loc[df[col_key]==key][col_name].values.item()
  return cval


## Languages

In [66]:
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages")
# merge pour récupérer des infos de référentiels
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","country_ref","flag_icon"]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid",], axis=1, inplace=True)
df_languages.rename(
    columns={"country_ref":"lang_country_ref",
             "flag_icon":"lang_flag_icon"},
    inplace=True)
# merge pour récupérer des infos de référentiels
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages_countries")
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","language_countries",]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid","lang_countries_vk",], axis=1, inplace=True)
df_languages.rename(
    columns={"language_countries":"lang_countries_vk",},
    inplace=True)
# Liste des colonnes pour le dict
vk_language_cols = [
    "lang_id",
    "lang_alpha3",
    "lang_alpha2_google",
    "lang_alpha2",
    "lang_wals",
    "lang_name_native",
    "lang_name_en",
    "lang_name_fr",
    "lang_is_available",
    "lang_is_tts",
    "lang_countries_vk",
    "lang_country_ref",
    "lang_flag_icon",
]

In [69]:
vk_lang = []
# pour chaque phrase : on itère
for vk_lang_tuple in vk_phrases_vals:
  # on transforme le tuple en dict
  vk_lang_dict = list(map(lambda x: dict(zip(vk_phrases_cols,x)), vk_lang_tuple))
  d_lang = {}
  #------L A N G U A G E S----------
  d_lang["language"] = list(dict.fromkeys(map(
      lambda x:x["phrase_language"],
      sorted(vk_lang_dict, key=lambda x: x["phrase_language"]))))[0]
  # get all language cols by key
  for field in vk_language_cols:
    d_lang[field] = df_languages.loc[df_languages["lang_id"]==d_lang["language"]][field].values.item()
  # ce vecteur est dans un format string, il faut le caster en array
  d_lang["lang_countries_vk"] = ast.literal_eval(d_lang["lang_countries_vk"])
  # pour chaque story
  vk_stories = []
  for el_story in list(dict.fromkeys(list(map(
      lambda x: x["phrase_related_story"],
      sorted(vk_lang_dict, key=lambda x: x["phrase_related_story"]))))):
    d_story = {}
    #------S T O R I E S----------
    d_story["story"] = el_story

    # pour chaque phrase
    vk_phrases = []
    for phrase_rec_id in list(dict.fromkeys(list(map(
        lambda x: x["phrase_rec_id"],
        filter(lambda d: d['phrase_related_story'] == el_story,
               sorted(vk_lang_dict, key=lambda x: x["phrase_paragraph"])))))):
      d_phrase = {}
      #------P H R A S E S----------
      # pour chaque colonne, on tranforme en dict
      for field in vk_phrases_cols:
        d_phrase[field] = f_phrases_list_to_dict_by_col(
            field,
            phrase_rec_id)
      d_phrase["phrase_rec_id"] = phrase_rec_id
      # rendons à story ce qui appartient à story...
      for field in vk_stories_cols:
        d_story[field] = d_phrase[field]
        d_phrase.pop(field)

      # pour chaque mot
      vk_words = []
      if len(d_phrase["phrase_words_tr_rec_id"])>0:
        list_words = ast.literal_eval(d_phrase["phrase_words_tr_rec_id"])
        for word in list_words:
          #------W O R D S----------
          d_word = {}
          # on récupère la valeur pour chaque attribut de mot
          for field in vk_word_cols:
            d_word[field] = f_words_get_value_by_col(
                df_words,
                "word_translation_id",
                field,
                word)

          d_word["word_tr_rec_id"] = word
          # lemmes et expressions
          vk_lem_exp = []
          # si lemmes : on itère le vecteur
          if len(d_word["word_lemmas_rec_id"])>0:
            list_lems = ast.literal_eval(d_word["word_lemmas_rec_id"])
            for lem in list_lems:
              #---K E Y W O R D S---
              d_lem = {}
              # on construit la clé de recherche id + langue
              lemkey = lem + "-" + d_word["word_language"]
              # pour chaque colonne (idem words) : on récupère la valeur à partir de la clé
              for field in vk_word_cols:
                d_lem[field] = f_words_get_value_by_col(
                  df_words,
                  "word_translation_id",
                  field,
                  lemkey)
              #----------------------
              d_lem["keyword"] = lem
              vk_lem_exp.append(d_lem)
              #----------------------

          #----------------------
          d_word["keywords"] = vk_lem_exp
          vk_words.append(d_word)
          #----------------------

      #----------------------
      d_phrase["words"] = vk_words
      vk_phrases.append(d_phrase)
      #----------------------

    #----------------------
    d_story["phrases"] = vk_phrases
    #----------------------
    vk_stories.append(d_story)


  #----------------------
  d_lang["stories"] = vk_stories
  #----------------------

  vk_lang.append(d_lang)


In [70]:
vk_lang[3]

{'language': 'ger',
 'lang_id': 'ger',
 'lang_alpha3': 'ger',
 'lang_alpha2_google': 'de',
 'lang_alpha2': 'de',
 'lang_wals': 'ger',
 'lang_name_native': 'Deutsch',
 'lang_name_en': 'German',
 'lang_name_fr': 'Allemand',
 'lang_is_available': True,
 'lang_is_tts': True,
 'lang_countries_vk': [{'country_iso3': 'aut',
   'country_iso2': 'AT',
   'country_name_fr': 'Autriche',
   'popularity': '100.00%',
   'popularity_as_float': 1.0,
   'speakers': '8,000,000',
   'is_official': 'TRUE',
   'official': 'Langue officielle',
   'national_flag': 'https://res.cloudinary.com/dhc7ovnwk/image/upload/v1670497510/langdeck/assets/flags_100px/at.png',
   'xy_coordinates': '47.3333,13.3333'},
  {'country_iso3': 'bel',
   'country_iso2': 'BE',
   'country_name_fr': 'Belgique',
   'popularity': '10.00%',
   'popularity_as_float': 0.1,
   'speakers': '1,200,000',
   'is_official': 'TRUE',
   'official': '',
   'national_flag': 'https://res.cloudinary.com/dhc7ovnwk/image/upload/v1670497510/langdeck/asse

## Sauvegarde au format JSON

In [71]:
with open('saynetes.txt', 'w') as f:
  json.dump(vk_lang, f, ensure_ascii=False)